In [1]:
# ---------------------------
# Import modules ------------
# ---------------------------
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# ---------------------------
# Initialize browser --------
# ---------------------------
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
#def scrape():
browser = init_browser()

In [4]:
# ---------------------------
# Retrieve headline & summary
# ---------------------------
url = "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(2)

In [5]:
mars_info = {}

In [6]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")
results = soup.find("li", class_="slide")
for r in results:
    try: 
        mars = {}
        mars_info["news_title"] = r.find("div", class_="content_title").get_text()
        mars_info["news_p"] = r.find("div", class_="article_teaser_body").get_text()
    except:
        break

In [7]:
mars_info

{'news_title': "NASA InSight's 'Mole' Is Out of Sight",
 'news_p': "Now that the heat probe is just below the Martian surface, InSight's arm will scoop some additional soil on top to help it keep digging so it can take Mars' temperature."}

In [8]:
# -----------------------
# Retrieve featured image
# -----------------------
img_site = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(img_site)
browser.links.find_by_partial_text('FULL').click()
browser.links.find_by_partial_href('/spaceimages/details').click()
browser.links.find_by_partial_href('/spaceimages/images/largesize/').click()

# https://splinter.readthedocs.io/en/latest/browser.html#verifying-page-url-with-browser-url
mars_info["featured_image_url"] = browser.url

In [9]:
# -----------------------
# Retrieve facts table --
# -----------------------
facts_url ='https://space-facts.com/mars/'
mars_facts = pd.read_html(facts_url)

facts_table = mars_facts[0]
mars_info["facts_table"] = facts_table.to_html(header=False, index=False, classes="table-striped")

In [10]:
# ------------------------------------------------
# Create dictionary for hemispheres and image urls
# ------------------------------------------------
hemisphere_image_urls = []
hemisphere_names = []

In [11]:
hems_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hems_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")
titles = soup.find_all("div", class_="description")
for t in titles:
    try:
        result = []
        result = t.find("h3").get_text()
        hemisphere_names.append(result)
    except:
        pass
        

In [12]:
hems_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hems_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")
titles = soup.find_all("div", class_="description")
for t in titles:
    try:
        result = []
        result = t.find("h3").get_text()
        hemisphere_names.append(result)
    except:
        pass

for h in hemisphere_names:
    hems_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(hems_url)
    time.sleep(1)
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    browser.links.find_by_partial_text(h).click()
        
    result = {}
    result["title"] = h
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    result["img_url"] = soup.find("div", class_="downloads").find("a")["href"]
    hemisphere_image_urls.append(result)


In [13]:
mars_info["cerberus"] = hemisphere_image_urls[0]
mars_info["schia"] = hemisphere_image_urls[1]
mars_info["syr"] = hemisphere_image_urls[2]
mars_info["valles"] = hemisphere_image_urls[3]
mars_info

{'news_title': "NASA InSight's 'Mole' Is Out of Sight",
 'news_p': "Now that the heat probe is just below the Martian surface, InSight's arm will scoop some additional soil on top to help it keep digging so it can take Mars' temperature.",
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17200_hires.jpg',
 'facts_table': '<table border="1" class="dataframe table-striped">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -

In [ ]:
browser.quit()